# Магические методы (часть 2)
## __call__, __getitem__, __setitem__, __iter__, __next__

In [ ]:
# __call__ - делает объект вызываемым (как функцию)

class Multiplier:
    """Класс, который умножает числа на заданный коэффициент"""
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, x):
        """Позволяет вызывать объект как функцию"""
        return x * self.factor

# Создаем объект
double = Multiplier(2)
triple = Multiplier(3)

# Вызываем объект как функцию
print(f"double(5) = {double(5)}")    # 10
print(f"triple(5) = {triple(5)}")    # 15
print(f"double(10) = {double(10)}")  # 20

# Проверим тип
print(f"Тип double: {type(double)}")  # <class '__main__.Multiplier'>
print(f"double вызываемый? {callable(double)}")  # True

# Более сложный пример: класс-счётчик
class Counter:
    def __init__(self, start=0):
        self.count = start
    
    def __call__(self):
        """При каждом вызове увеличивает счётчик и возвращает его значение"""
        self.count += 1
        return self.count

# Использование
counter = Counter()
print(f"counter() = {counter()}")  # 1
print(f"counter() = {counter()}")  # 2
print(f"counter() = {counter()}")  # 3

## __call__ - объект как функция

Метод `__call__` позволяет экземплярам класса вести себя как функции. После его определения объекты класса можно "вызывать" с помощью круглых скобок и аргументов.

**Основные применения:**
1. Объекты с состоянием, которые нужно периодически вызывать
2. Замена замыканий (closures) с более сложной логикой
3. Создание декораторов через классы
4. Функциональные объекты с памятью (как в примере со счётчиком)

In [ ]:
# __getitem__ и __setitem__ - доступ по индексу/ключу

class SmartList:
    """Класс-список с дополнительной функциональностью"""
    def __init__(self, *args):
        self._data = list(args)
    
    def __getitem__(self, index):
        """Получение элемента по индексу или срезу"""
        return self._data[index]
    
    def __setitem__(self, index, value):
        """Установка элемента по индексу"""
        self._data[index] = value
    
    def __len__(self):
        return len(self._data)
    
    def __repr__(self):
        return f"SmartList({self._data})"

# Создаем объект
my_list = SmartList(1, 2, 3, 4, 5)

# Используем __getitem__
print(f"my_list[0] = {my_list[0]}")        # 1
print(f"my_list[2] = {my_list[2]}")        # 3
print(f"my_list[-1] = {my_list[-1]}")      # 5 (отрицательные индексы)
print(f"my_list[1:4] = {my_list[1:4]}")    # [2, 3, 4] (срезы!)

# Используем __setitem__
my_list[0] = 100
my_list[2] = 300
print(f"После изменения: {my_list}")  # SmartList([100, 2, 300, 4, 5])

# Проверяем границы
try:
    print(my_list[10])  # Выход за границы
except IndexError as e:
    print(f"Ошибка: {e}")

# Класс для работы со словарём
class CaseInsensitiveDict:
    """Словарь, не чувствительный к регистру ключей"""
    def __init__(self):
        self._data = {}
    
    def __getitem__(self, key):
        """Получение значения по ключу (регистронезависимо)"""
        return self._data[key.lower()]
    
    def __setitem__(self, key, value):
        """Установка значения по ключу (регистронезависимо)"""
        self._data[key.lower()] = value
    
    def __repr__(self):
        return f"CaseInsensitiveDict({self._data})"

# Использование
config = CaseInsensitiveDict()
config['ServerName'] = 'MainServer'
config['PORT'] = 8080

print(f"config['servername'] = {config['servername']}")  # MainServer
print(f"config['port'] = {config['port']}")              # 8080

config['Port'] = 9090  # Изменит существующий ключ
print(f"После изменения: {config}")

## __getitem__ и __setitem__ - индексирование объектов

Эти методы позволяют использовать квадратные скобки `[]` для доступа к элементам объекта, как в списках и словарях.

**`__getitem__(self, key)`** - вызывается при получении значения: `obj[key]`
**`__setitem__(self, key, value)`** - вызывается при установке значения: `obj[key] = value`

**Особенности:**
- `key` может быть не только целым числом, но и строкой (для словарей), срезом (slice) или другим типом
- Python автоматически передает срезы как объекты `slice(start, stop, step)`
- Для поддержки отрицательных индексов нужно реализовать логику самостоятельно или использовать встроенные типы

In [ ]:
# __iter__ и __next__ - создание итераторов

class Range:
    """Класс, создающий последовательность чисел (как range)"""
    def __init__(self, start, stop, step=1):
        self.start = start
        self.stop = stop
        self.step = step
        self.current = start
    
    def __iter__(self):
        """Возвращает сам объект как итератор"""
        self.current = self.start  # Сбрасываем состояние
        return self
    
    def __next__(self):
        """Возвращает следующее значение или вызывает StopIteration"""
        if (self.step > 0 and self.current >= self.stop) or \
           (self.step < 0 and self.current <= self.stop):
            raise StopIteration
        
        value = self.current
        self.current += self.step
        return value

# Использование как итератора
print("Итерация по Range(0, 5):")
for i in Range(0, 5):
    print(i, end=" ")  # 0 1 2 3 4
print()

# Итерация вручную
r = Range(5, 0, -1)
iterator = iter(r)  # Вызывает __iter__
print("\nРучная итерация:")
try:
    while True:
        print(next(iterator), end=" ")  # 5 4 3 2 1
except StopIteration:
    print("\nИтерация завершена")

# Более сложный пример: итератор по четным числам
class EvenNumbers:
    """Итератор, возвращающий четные числа до предела"""
    def __init__(self, limit):
        self.limit = limit
        self.current = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current >= self.limit:
            raise StopIteration
        
        value = self.current
        self.current += 2
        return value

print("\nЧетные числа до 10:")
for num in EvenNumbers(10):
    print(num, end=" ")  # 0 2 4 6 8
print()

In [ ]:
# Отдельный класс для итератора (более правильный подход)

class Fibonacci:
    """Класс, создающий итератор для чисел Фибоначчи"""
    def __init__(self, max_count):
        self.max_count = max_count
    
    def __iter__(self):
        """Возвращает отдельный объект-итератор"""
        return FibonacciIterator(self.max_count)

class FibonacciIterator:
    """Отдельный класс для итерации"""
    def __init__(self, max_count):
        self.max_count = max_count
        self.count = 0
        self.a, self.b = 0, 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.count >= self.max_count:
            raise StopIteration
        
        if self.count == 0:
            result = self.a
        elif self.count == 1:
            result = self.b
        else:
            self.a, self.b = self.b, self.a + self.b
            result = self.b
        
        self.count += 1
        return result

# Использование
print("Первые 10 чисел Фибоначчи:")
for i, num in enumerate(Fibonacci(10)):
    print(f"F{i} = {num}")

# Второй проход по тому же объекту
fib = Fibonacci(5)
print("\nПервые 5 чисел Фибоначчи (дважды):")
print("Первый проход:", list(fib))  # [0, 1, 1, 2, 3]
print("Второй проход:", list(fib))  # [0, 1, 1, 2, 3] - работает корректно!

## __iter__ и __next__ - создание итераторов

Итераторы - объекты, которые позволяют последовательно получать элементы. Для создания итератора нужно реализовать два метода:

**`__iter__(self)`** - должен возвращать объект-итератор (обычно `self`)
**`__next__(self)`** - должен возвращать следующий элемент или вызывать `StopIteration`

**Разделение на коллекцию и итератор (рекомендуемый подход):**
- Класс коллекции реализует только `__iter__`, возвращая новый итератор
- Отдельный класс итератора реализует `__iter__` (возвращает `self`) и `__next__`

**Преимущества разделения:**
1. Можно итерироваться по одному объекту несколько раз одновременно
2. Состояние итерации изолировано от коллекции
3. Более чистая архитектура

In [ ]:
# Комбинированный пример: матрица с поддержкой всех методов

class Matrix:
    """Класс для работы с матрицами, поддерживающий индексацию и итерацию"""
    def __init__(self, rows, cols, fill_value=0):
        self.rows = rows
        self.cols = cols
        self.data = [[fill_value for _ in range(cols)] for _ in range(rows)]
    
    def __getitem__(self, index):
        """Поддержка: matrix[row][col] и matrix[row, col]"""
        if isinstance(index, tuple) and len(index) == 2:
            # Обращение matrix[row, col]
            row, col = index
            return self.data[row][col]
        else:
            # Обращение matrix[row] - возвращаем строку
            return self.data[index]
    
    def __setitem__(self, index, value):
        """Установка значения: matrix[row, col] = value"""
        if isinstance(index, tuple) and len(index) == 2:
            row, col = index
            self.data[row][col] = value
        else:
            # Если передана только строка, заменяем всю строку
            self.data[index] = value
    
    def __iter__(self):
        """Итерация по строкам матрицы"""
        return MatrixIterator(self)
    
    def __call__(self, scalar):
        """Умножение матрицы на скаляр: matrix(5)"""
        result = Matrix(self.rows, self.cols)
        for i in range(self.rows):
            for j in range(self.cols):
                result[i, j] = self[i, j] * scalar
        return result
    
    def __repr__(self):
        return f"Matrix({self.rows}x{self.cols})"

class MatrixIterator:
    """Итератор по строкам матрицы"""
    def __init__(self, matrix):
        self.matrix = matrix
        self.row = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.row >= self.matrix.rows:
            raise StopIteration
        row_data = self.matrix[self.row]
        self.row += 1
        return row_data

# Создаем и используем матрицу
m = Matrix(3, 3)
print(f"Пустая матрица: {m}")

# Заполняем значениями
for i in range(3):
    for j in range(3):
        m[i, j] = i * 3 + j + 1

print("\nЗаполненная матрица:")
for row in m:  # Используем __iter__
    print(row)

# Доступ по индексам
print(f"\nm[0, 1] = {m[0, 1]}")  # 2
print(f"m[2] = {m[2]}")          # [7, 8, 9] - вся строка

# Изменение значений
m[1, 1] = 100
print(f"\nПосле изменения m[1, 1]: {m[1, 1]}")

# Умножение на скаляр через __call__
m2 = m(2)  # Умножаем все элементы на 2
print("\nМатрица умноженная на 2:")
for row in m2:
    print(row)

In [ ]:
# Пример с генератором в __iter__ (альтернативный подход)

class SquareNumbers:
    """Класс, возвращающий квадраты чисел"""
    def __init__(self, limit):
        self.limit = limit
    
    def __iter__(self):
        """Используем генератор вместо отдельного класса-итератора"""
        for i in range(self.limit):
            yield i ** 2
    
    def __getitem__(self, index):
        """Прямой доступ к квадрату по индексу"""
        if 0 <= index < self.limit:
            return index ** 2
        raise IndexError("Индекс вне диапазона")

# Использование
squares = SquareNumbers(5)

print("Квадраты через итерацию:")
for square in squares:
    print(square, end=" ")  # 0 1 4 9 16
print()

print("\nПрямой доступ:")
print(f"squares[0] = {squares[0]}")    # 0
print(f"squares[3] = {squares[3]}")    # 9

# Итерируемся несколько раз
print("\nДве итерации подряд:")
print("Первая:", list(squares))  # [0, 1, 4, 9, 16]
print("Вторая:", list(squares))  # [0, 1, 4, 9, 16] - работает!

In [ ]:
# Декоратор как класс с __call__

class Timer:
    """Декоратор для измерения времени выполнения функции"""
    def __init__(self, func):
        self.func = func
        self.call_count = 0
    
    def __call__(self, *args, **kwargs):
        """Вызывается при вызове декорированной функции"""
        import time
        self.call_count += 1
        start_time = time.time()
        result = self.func(*args, **kwargs)
        end_time = time.time()
        
        print(f"{self.func.__name__} вызвана {self.call_count} раз")
        print(f"Время выполнения: {end_time - start_time:.6f} секунд")
        return result

# Применение декоратора
@Timer
def slow_function(n):
    """Функция, которая выполняется некоторое время"""
    import time
    time.sleep(0.5)  # Имитация долгого выполнения
    return n * 2

# Вызов декорированной функции
print("Первый вызов:")
result1 = slow_function(5)
print(f"Результат: {result1}")

print("\nВторой вызов:")
result2 = slow_function(10)
print(f"Результат: {result2}")

# Декоратор с параметрами
class Repeat:
    """Декоратор с параметрами"""
    def __init__(self, times):
        self.times = times
    
    def __call__(self, func):
        """Принимает функцию и возвращает обёрнутую"""
        def wrapper(*args, **kwargs):
            results = []
            for i in range(self.times):
                print(f"Вызов {i+1}/{self.times}")
                result = func(*args, **kwargs)
                results.append(result)
            return results[-1] if results else None
        return wrapper

@Repeat(times=3)
def greet(name):
    print(f"Привет, {name}!")
    return f"Приветствие для {name} выполнено"

print("\nДекоратор с параметрами:")
result = greet("Анна")
print(f"Итоговый результат: {result}")

## Практические применения

### __call__:
1. **Декораторы классов** - когда нужны параметры или состояние
2. **Функторы (функциональные объекты)** - объекты с состоянием, которые ведут себя как функции
3. **Эмуляция функций** - когда нужна сложная инициализация перед вызовом

### __getitem__/__setitem__:
1. **Кастомные коллекции** - списки, словари, матрицы с особой логикой
2. **Прокси-объекты** - доступ к данным с преобразованием
3. **Интерфейс к внешним данным** - базы данных, файлы, сетевые ресурсы

### __iter__/__next__:
1. **Ленивые вычисления** - генерация данных по требованию
2. **Стриминг данных** - обработка больших объемов без загрузки в память
3. **Кастомные последовательности** - с особой логикой итерации

## Важные моменты

1. **Итераторы vs Итерируемые объекты**:
   - Итерируемый объект имеет `__iter__` и возвращает итератор
   - Итератор имеет `__next__` (и обычно `__iter__`, возвращающий `self`)

2. **Протокол итератора**:
   - `iter(obj)` вызывает `obj.__iter__()`
   - `next(iterator)` вызывает `iterator.__next__()`
   - Цикл `for` автоматически использует эти функции

3. **Протокол последовательности**:
   - Для полной поддержки индексации также полезны `__len__`, `__contains__`
   - Срезы передаются как объекты `slice(start, stop, step)`

4. **__call__ и callable()**:
   - Функция `callable(obj)` возвращает `True` если объект можно вызвать
   - Любой объект с `__call__` считается вызываемым

In [ ]:
# Финальный пример: умная коллекция с поддержкой всех методов

class SmartCollection:
    """Коллекция, поддерживающая индексацию, итерацию и вызов"""
    def __init__(self, *items):
        self.items = list(items)
        self.access_count = 0
    
    # Индексация
    def __getitem__(self, index):
        self.access_count += 1
        if isinstance(index, slice):
            return SmartCollection(*self.items[index])
        return self.items[index]
    
    def __setitem__(self, index, value):
        self.items[index] = value
    
    def __len__(self):
        return len(self.items)
    
    # Итерация
    def __iter__(self):
        return iter(self.items)  # Используем встроенный итератор списка
    
    # Вызов как функции
    def __call__(self, func):
        """Применяет функцию ко всем элементам и возвращает новую коллекцию"""
        results = [func(item) for item in self.items]
        return SmartCollection(*results)
    
    # Представление
    def __str__(self):
        return f"SmartCollection{tuple(self.items)}"
    
    def __repr__(self):
        return f"SmartCollection({', '.join(repr(x) for x in self.items)})"

# Создаем коллекцию
collection = SmartCollection(1, 2, 3, 4, 5)
print(f"Исходная коллекция: {collection}")
print(f"Длина: {len(collection)}")

# Индексация
print(f"\ncollection[0] = {collection[0]}")
print(f"collection[1:4] = {collection[1:4]}")
print(f"Счётчик обращений: {collection.access_count}")

# Итерация
print("\nИтерация:")
for item in collection:
    print(item, end=" ")
print()

# Вызов как функции
def double(x):
    return x * 2

def square(x):
    return x ** 2

doubled = collection(double)
print(f"\nУдвоенные: {doubled}")

squared = collection(square)
print(f"Квадраты: {squared}")

# Цепочка вызовов
result = collection(double)(square)
print(f"Удвоить, затем возвести в квадрат: {result}")

## Итоги

### __call__:
- Делает объект вызываемым как функцию
- Полезен для декораторов, функторов, объектов с состоянием

### __getitem__ и __setitem__:
- Позволяют использовать квадратные скобки для доступа
- Поддерживают индексы, срезы, ключи
- Основа для создания кастомных коллекций

### __iter__ и __next__:
- Определяют поведение при итерации
- Позволяют создавать ленивые вычисления
- Разделение на коллекцию и итератор - хорошая практика

**Сочетание этих методов** позволяет создавать объекты, которые ведут себя как встроенные типы Python, но с кастомной логикой. Это мощный инструмент для создания чистого, понятного API.